<a href="https://colab.research.google.com/github/mmanam2/mmanam2/blob/main/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculates the PRIEST COVID-19 Clinical Severity Score based on patient input values.

    The PRIEST score is used to assess the severity of COVID-19 symptoms and predict the probability of a severe outcome.
    A score between 17 and 25 indicates a probability of 0.59 for severe outcomes, while a score of 26 or higher
    indicates a probability of 0.99 for severe outcomes.

    Parameters:
    -----------
    sex (str): Gender assigned at birth ('male', 'female').
    age (int): Age in years.
    respiratory_rate (int): Respiratory rate in breaths per minute.
    oxygen_saturation (float): Oxygen saturation as a decimal between 0 and 1 (e.g. 0.9 for 90%).
    heart_rate (int): Heart rate in beats per minute.
    systolic_bp (int): Systolic blood pressure in mmHg.
    temperature (float): Body temperature in Celsius.
    alertness (str): Alertness status ('alert', 'confused', 'not alert').
    inspired_oxygen (str): Description of inspired oxygen therapy (e.g., 'none', 'supplemental oxygen').
    performance_status (str): Physical performance status (e.g., 'normal', 'limited activity', 'bedbound').

    Returns:
    --------
    float: Probability of severe outcome (0.59 or 0.99 based on the calculated score).
    None: If invalid inputs (e.g., sex is not 'male' or 'female').

    Example:
    --------
    >>> priest("mAle", 81,25,0.91,131,220,39.1,"Confused","Supplemental oxygen","Bed/chair bound, no self-care")
    0.88
    >>> priest("mAlje", 81,25,0.91,131,220,39.1,"Confused","Supplemental oxygen","Bed/chair bound, no self-care")
    'None'
    >>> priest("FEMALE", 80,22,0.96,41,115,38.1,"Alert","AIR","limited activity, can self-care")
    0.26

    """
    sex=sex.lower()
    alertness=alertness.lower()
    inspired_oxygen=inspired_oxygen.lower()
    performance_status=performance_status.lower()
    score=0
    if sex not in ['male', 'female'] :
        return 'None'
    # Sex check
    if sex.lower() == 'male':
        score += 1

    # Age check
    if 16 <= age <= 49:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    else:
        score += 4

    # Respiratory rate check
    if respiratory_rate < 9:
        score += 3
    elif 9 <= respiratory_rate <= 11:
        score += 1
    elif 12 <= respiratory_rate <= 20:
        score += 0
    elif 21 <= respiratory_rate <= 24:
        score += 2
    else:
        score += 3

    # Oxygen saturation check
    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif 0.92 <= oxygen_saturation <= 0.93:
        score += 2
    else:
        score += 3

    # Heart rate check
    if heart_rate < 41:
        score += 3
    elif 41 <= heart_rate <= 50:
        score += 1
    elif 51 <= heart_rate <= 90:
        score += 0
    elif 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    else:
        score += 3

    # Systolic BP check
    if systolic_bp < 91:
        score += 3
    elif 91 <= systolic_bp <= 100:
        score += 2
    elif 101 <= systolic_bp <= 110:
        score += 1
    elif 111 <= systolic_bp <= 219:
        score += 0
    else:
        score += 3

    # Temperature check
    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36.0:
        score += 1
    elif 36.1 <= temperature <= 38.0:
        score += 0
    elif 38.1 <= temperature <= 39.0:
        score += 1
    else:
        score += 2

    # Alertness check
    if alertness.lower() == "confused or not alert".lower():
        score += 3
    elif alertness.lower() == "alert".lower():
        score += 0

    # Inspired oxygen check
    if inspired_oxygen.lower() == "supplemental oxygen".lower():
        score += 2

    # Performance status check
    if performance_status.lower() == "limited strenuous activity, can do light activity".lower():
        score += 1
    elif performance_status.lower() == "limited activity, can self-care".lower():
        score += 2
    elif performance_status.lower() == "limited self-care".lower():
        score += 3
    elif performance_status.lower() == "bed/chair bound, no self-care".lower():
        score += 4

    # j is risk percentage in range (0,1)
    probability=[(1,0.01),(3,0.02),(4,0.03),(5,0.9),(6,0.15),(7,0.18),(8,0.22),(9,0.26),(10,0.29),(11,0.34),(12,0.38),(13,0.46),(14,0.47),(15,0.49),(16,0.55),(25,0.88),(float('inf'),0.99)]
    for l,j  in probability:
            if score <= l:
                return j

In [2]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName
Trying:
    priest("mAle", 81,25,0.91,131,220,39.1,"Confused","Supplemental oxygen","Bed/chair bound, no self-care")
Expecting:
    0.88
ok
Trying:
    priest("mAlje", 81,25,0.91,131,220,39.1,"Confused","Supplemental oxygen","Bed/chair bound, no self-care")
Expecting:
    'None'
ok
Trying:
    priest("FEMALE", 80,22,0.96,41,115,38.1,"Alert","AIR","limited activity, can self-care")
Expecting:
    0.26
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [3]:
import requests
def find_hospital(age, sex, risk_pct):
    """
    Calls the hospital web service to find the appropriate hospital for the patient.

    Parameters:
    age (int): Age of the patient
    sex (str): Gender assigned at birth ('male', 'female')
    risk_pct (float): The patient's risk percentage (between 0 and 1)

    Returns:
    str: The name of the hospital or None if an error occurs

    Example Test Cases:
    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(30, 'female', 0.2)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital(80, 'male', 0.9)
    'Wesley Woods Geriatric Hospital'
    """
    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    params = {
        "age": age,
        "sex": sex.lower(),
        "risk_pct": risk_pct
    }

    try:
        # Make the GET request
        response = requests.get(base_url, params=params)

        # Check if the response was successful
        if response.status_code == 200:
            data = response.json()
            return data.get("hospital", None)  # Return hospital name if exists
        else:
            return None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [4]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(30, 'female', 0.2)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(80, 'male', 0.9)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [5]:
import requests
import json

def get_address(hospital_name):
    """
    Fetches the address of the hospital based on its name from a JSON file.

    Parameters:
    hospital_name (str): The name of the hospital to look up.

    Returns:
    str: The address of the hospital if found.
    None: If the hospital is not found in the JSON file.

    Example Test Cases:

    >>> get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL')
    '417 THIRD AVENUE'

    >>> get_address('AU MEDICAL CENTER')
    '1120 15TH STREET'

    >>> get_address('Nonexistent ospital')
    'None'
    """

    # URL to download the JSON file
    url = 'https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-'

    try:
        # Fetch the JSON data using requests
        address_response = requests.get(url)

        # Raise an error if the request failed
        address_response.raise_for_status()

        # Parse the JSON data
        address_data = address_response.json()


        if hospital_name.upper() in address_data:
            return address_data[hospital_name.upper()]['ADDRESS']
        else:
            return 'None'

    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return 'None'
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error parsing JSON or missing data: {e}")
        return 'None'
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL')
Expecting:
    '417 THIRD AVENUE'
ok
Trying:
    get_address('AU MEDICAL CENTER')
Expecting:
    '1120 15TH STREET'
ok
Trying:
    get_address('Nonexistent ospital')
Expecting:
    'None'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [6]:
def process_people(file_url):
    """
    Process the pipe-delimited people.psv file to calculate risk scores and assign hospitals.

    Parameters:
    file_url (str): The URL of the file to be processed.

    Returns:
    list: A list of dictionaries containing patient details and their assigned hospital and address.
    """
    try:
        # Fetch the file data using requests
        people_response = requests.get(file_url)
        people_response.raise_for_status()
        # print(people_response,"ok")

        # Split the content by newlines to get each row
        content = people_response.text.strip().split('\n')
        # print(content)




        people_data = []
        # Process the rows (skip the header row)
        for row in content[1:]:
            # Removing the pipe delimited and making a list for each patient which is "patient_data"
            patient_data = row.split('|')
            # print(patient_data)

            # Extract relevant data for PRIEST calculation
            patient_number = patient_data[0]
            sex = patient_data[1]
            age = int(patient_data[2])
            breath = int(patient_data[3])
            o2sat = float(patient_data[4])
            heart = int(patient_data[5])
            systolic = int(patient_data[6])
            temp = float(patient_data[7])
            alertness = patient_data[8]
            inspired = patient_data[9]
            status = patient_data[10]

            # Call the function you implemented for risk calculation (assumed to be calculate_risk)
            risk_percentage = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)
            # print(risk_percentage)

            # Based on the risk percentage, determine the appropriate hospital
            hospital = find_hospital(age, sex, risk_percentage)

            if hospital is not None:
                address = get_address(hospital)
            else:
                address = 'Unknown'
            # Store the processed data in the list
            people_data.append({
                patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status,risk_percentage, hospital, address]
            })

        return people_data

    except requests.RequestException as e:
        print(f"An error occurred while fetching the file: {e}")
        return []
    except Exception as e:
        print(f"An error occurred during processing: {e}")
        return []
people_csv_url = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'
# process_people(people_csv_url)

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [7]:
people_results_url="https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
people_results_response = requests.get(people_results_url)
people_results_json_data = people_results_response.json()

# Compare local results with fetched JSON
def compare_data(local, online):
    for local_patient in local:
        for patient_id, local_details in local_patient.items():
            # Get corresponding patient data from the online JSON
            online_details = online.get(patient_id)

            if online_details:
                # Perform comparison
                if local_details == online_details:  # Ignore the risk score if needed
                    print(f"Patient {patient_id} details match.")
                else:
                    print(f"Patient {patient_id} details do NOT match.")
            else:
                print(f"Patient {patient_id} not found in online data.")

# Call the function to compare
my_results=process_people(people_csv_url)
compare_data(my_results, people_results_json_data)

Patient E9559 details match.
Patient E9385 details match.
Patient E3067 details match.
Patient E9422 details match.
Patient E8661 details match.
Patient E6235 details match.
Patient E4451 details match.
Patient E8433 details match.
Patient E7593 details match.
Patient E3296 details match.
Patient E4157 details match.
Patient E7702 details match.
Patient E8158 details match.
Patient E4795 details match.
Patient E8800 details match.
Patient E4855 details match.
Patient E9691 details match.
Patient E4535 details match.
Patient E2360 details match.
Patient E3447 details match.
Patient E8208 details match.
Patient E4428 details do NOT match.
Patient E2766 details match.
Patient E3888 details match.
Patient E7110 details match.
Patient E2668 details do NOT match.
Patient E8482 details match.
Patient E9619 details match.
Patient E1700 details match.
Patient E8585 details match.
Patient E8633 details match.
Patient E2117 details match.
Patient E2322 details match.
Patient E6912 details match.


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---